In [ ]:
import os
import cv2
import random
import numpy as np
from zipfile import ZipFile
from PIL import UnidentifiedImageError

In [ ]:
file_name = "/content/Skin_ Conditions.zip"

with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('Done')

Done


In [ ]:
base_path = '/content/Skin_ Conditions'
categories = ['Acne', 'Carcinoma', 'Eczema', 'Keratosis', 'Milia', 'Rosacea']
target_count = 400

In [ ]:
def rotate_image(image):
    angle = random.randint(-30, 30)
    h, w = image.shape[:2]
    M = cv2.getRotationMatrix2D((w / 2, h / 2), angle, 1)
    return cv2.warpAffine(image, M, (w, h))

In [ ]:
def flip_image(image):
    return cv2.flip(image, random.choice([-1, 0, 1]))

In [ ]:
def scale_image(image):
    scale = random.uniform(0.7, 1.3)
    h, w = image.shape[:2]
    return cv2.resize(image, (int(w * scale), int(h * scale)))

In [ ]:
def crop_image(image):
    h, w = image.shape[:2]
    crop_size = random.uniform(0.7, 1.0)
    x = random.randint(0, int(w * (1 - crop_size)))
    y = random.randint(0, int(h * (1 - crop_size)))
    return image[y:int(y + h * crop_size), x:int(x + w * crop_size)]

In [ ]:
def augment_image(image):
    augmentations = [rotate_image, flip_image, scale_image, crop_image]
    augmentation = random.choice(augmentations)
    return augmentation(image)

In [ ]:
augmented_path = 'Augmented_Skin_Conditions'

os.makedirs(augmented_path, exist_ok=True)
for category in categories:
    os.makedirs(os.path.join(augmented_path, category), exist_ok=True)

In [ ]:
for category in categories:
    images_path = os.path.join(base_path, category)
    images = os.listdir(images_path)
    num_images = len(images)
    image_counter = 1

    def get_unique_name(category, image_counter, ext):
        while True:
            filename = f"{category}_{image_counter}{ext}"
            filepath = os.path.join(augmented_path, category, filename)
            if not os.path.exists(filepath):
                return filename
            image_counter += 1

    if num_images >= target_count:
        for img in images[:target_count]:
            try:
                image = cv2.imread(os.path.join(images_path, img))
                if image is None:
                    continue
                ext = os.path.splitext(img)[1]
                augmented_image_name = get_unique_name(category, image_counter, ext)
                augmented_image_path = os.path.join(augmented_path, category, augmented_image_name)
                cv2.imwrite(augmented_image_path, image)
                image_counter += 1
            except Exception as e:
                print(f"Error processing image {img}: {e}")
    else:
        for img in images:
            try:
                image = cv2.imread(os.path.join(images_path, img))
                if image is None:
                    continue
                ext = os.path.splitext(img)[1]
                augmented_image_name = get_unique_name(category, image_counter, ext)
                augmented_image_path = os.path.join(augmented_path, category, augmented_image_name)
                cv2.imwrite(augmented_image_path, image)
                image_counter += 1
            except Exception as e:
                print(f"Error processing image {img}: {e}")

        i = 0
        while num_images < target_count:
            try:
                image_file = images[i % len(images)]
                image_path = os.path.join(images_path, image_file)
                image = cv2.imread(image_path)
                if image is None:
                    i += 1
                    continue

                augmented_image = augment_image(image)
                ext = os.path.splitext(image_file)[1]
                augmented_image_name = get_unique_name(category, image_counter, ext)
                augmented_image_path = os.path.join(augmented_path, category, augmented_image_name)

                cv2.imwrite(augmented_image_path, augmented_image)
                num_images += 1
                image_counter += 1
                i += 1
            except Exception as e:
                print(f"Error processing image {image_file}: {e}")
                i += 1

print("Augmentation complete!")

Augmentation complete!


In [ ]:
import shutil
from google.colab import files

folder_to_zip = '/content/Augmented_Skin_Conditions'
output_filename = '/content/Skin_Conditions_Augmented.zip'

shutil.make_archive(output_filename.replace('.zip', ''), 'zip', folder_to_zip)

files.download(output_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>